In [1]:
#import libraries
import numpy as np
from scipy.spatial.transform import Rotation
from scipy.sparse import csr_matrix

In [2]:
#Input orbital parameters and constants
#a=7016 #km
h=  19646.8827043885  #km^2/s^2
e=  0.947540967471404
i=  124.04786296943432  #degrees
Omega=  190.6196552761551  #degrees
omega=  303.09103460599  #degrees
theta=  159.6116163264222 #degrees
#gravitational parameter
mu = 3.986e5 # km^3/s^2

In [3]:
#compute h from a and e
#h=np.sqrt(a*mu*(1 - e ** 2))

In [4]:
#Convert angles from degrees to radians
convFactor=
i=i*np.pi/180.
Omega=Omega*np.pi/180.
omega=omega*np.pi/180.
theta=theta*np.pi/180.
print('i=',i,'radians')
print('Omega=',Omega)
print('omega=',omega)
print('theta=',theta)

i= 2.165043638879379 radians
Omega= 3.3269406035854874
omega= 5.2899364871506
theta= 2.785748229326004


In [5]:
#Define postion and velocity vectors in the perifocal frame from h,e, theta
r_p=h**2/mu/(1+e*np.cos(theta))*np.array((np.cos(theta),np.sin(theta),0))
v_p=mu / h * np.array((-np.sin(theta), e + np.cos(theta),0))
print("r_p=",r_p)
print("v_p=",v_p)

r_p= [-8117.71199255  3017.07673187     0.        ]
v_p= [-7.06804598  0.20670287  0.        ]


In [6]:
#Define Rotation matrices
#First one about Z axis an angle Omega
R3=np.array(((np.cos(Omega),np.sin(Omega),0),(-np.sin(Omega),np.cos(Omega),0),(0,0,1)))
R2=np.array(((1,0,0),(0,np.cos(i),np.sin(i)),(0,-np.sin(i),-np.cos(i))))
R1=np.array(((np.cos(omega),np.sin(omega),0),(-np.sin(omega),np.cos(omega),0),(0,0,1)))
print("R1=",R1)
print("R2=",R2)
print("R3=",R3)

R1= [[ 0.54597087 -0.83780416  0.        ]
 [ 0.83780416  0.54597087  0.        ]
 [ 0.          0.          1.        ]]
R2= [[ 1.          0.          0.        ]
 [ 0.         -0.55988526  0.82857015]
 [ 0.         -0.82857015  0.55988526]]
R3= [[-0.98287219 -0.18428854  0.        ]
 [ 0.18428854 -0.98287219  0.        ]
 [ 0.          0.          1.        ]]


In [7]:
#Multiply matrices to obtain QXx=R1R2R3
QXx=R1@R2@R3
print("QXx=",QXx)

QXx= [[-0.45017459 -0.56165615 -0.69417952]
 [-0.87978792  0.14604769  0.45237517]
 [-0.15269598  0.81437856  0.55988526]]


In [8]:
#Compute the QxX transposing QXx
QxX=np.transpose(QXx)
print("QxX=",QxX)

QxX= [[-0.45017459 -0.87978792 -0.15269598]
 [-0.56165615  0.14604769  0.81437856]
 [-0.69417952  0.45237517  0.55988526]]


In [21]:
#To follow the algorithm we defined in class,
#we need to use the column vector, this is the main difference
#in the matrix multiplication

#reshape, allow to convert the column vector we defined for the 
#perifocal frame into a column. The (-1,1) do the conversion from 
#row to column; if you need from column to row, use reshape with (1,-1)
r_pc=r_p.reshape(-1,1)
#We perform the matrix computation as always
r=QxX@r_pc
print("r=",r," km")
print("r=",r.reshape(1,-1), " km")
#convert the perifocal velocity from row to column vector 
v_pc=v_p.reshape(-1,1)
#perform the multiplication as always
v=QxX@v_pc
print("v=",v, " km/s")
print("v=",v.reshape(1,-1), " km/s")

r= [[1000.]
 [5000.]
 [7000.]]  km
r= [[1000. 5000. 7000.]]  km
v= [[3.]
 [4.]
 [5.]]  km/s
v= [[3. 4. 5.]]  km/s


In [10]:
R=Rotation.from_euler("ZXZ",[-omega,-i,-Omega])
print("R=",R.as_matrix())

R= [[-0.45017459 -0.56165615 -0.69417952]
 [-0.87978792  0.14604769  0.45237517]
 [-0.15269598  0.81437856 -0.55988526]]


In [11]:
r=r_p@R.as_matrix()
print(r," km")

[1000. 5000. 7000.]


In [12]:
v=v_p@R.as_matrix()
print(v, " km/s")

[3. 4. 5.]


In [16]:
r=r_p@QXx
print(r," km")
v=v_p@QXx
print(v," km/s")

[1000. 5000. 7000.]  km
[3. 4. 5.]  km/s
